In [34]:
import os
import json
from typing import Dict, List, Optional, Union, cast

from env import github_token, github_username

from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import re
import requests

# TODO: Make a github personal access token.
#     1. Go here and generate a personal access token: https://github.com/settings/tokens
#        You do _not_ need select any scopes, i.e. leave all the checkboxes unchecked
#     2. Save it in your env.py file under the variable `github_token`
# TODO: Add your github username to your env.py file under the variable `github_username`
# TODO: Add more repositories to the `REPOS` list below.

def web_scrape_repos():
    
    search_topics = "https://github.com/search?p="

    REPOS = []
    
    for page in range(1, 20):

        req = requests.get(search_topics + str(page) + "&q=" + 'bitcoin' + "&type=Repositories")
        soup = BeautifulSoup(req.text, 'html.parser')

        repos = soup.find_all('a', class_='v-align-middle')
        for link in repos:
            REPOS.append(link['href'][1:])
    
    return REPOS

REPOS = web_scrape_repos()

headers = {"Authorization": f"token {github_token}", "User-Agent": github_username}

if headers["Authorization"] == "token " or headers["User-Agent"] == "":
    raise Exception(
        "You need to follow the instructions marked TODO in this script before trying to use it"
    )


def github_api_request(url: str) -> Union[List, Dict]:
    response = requests.get(url, headers=headers)
    response_data = response.json()
    if response.status_code != 200:
        raise Exception(
            f"Error response from github api! status code: {response.status_code}, "
            f"response: {json.dumps(response_data)}"
        )
    return response_data


def get_repo_language(repo: str) -> str:
    url = f"https://api.github.com/repos/{repo}"
    repo_info = github_api_request(url)
    if type(repo_info) is dict:
        repo_info = cast(Dict, repo_info)
        if "language" not in repo_info:
            raise Exception(
                "'language' key not round in response\n{}".format(json.dumps(repo_info))
            )
        return repo_info["language"]
    raise Exception(
        f"Expecting a dictionary response from {url}, instead got {json.dumps(repo_info)}"
    )


def get_repo_contents(repo: str) -> List[Dict[str, str]]:
    url = f"https://api.github.com/repos/{repo}/contents/"
    contents = github_api_request(url)
    if type(contents) is list:
        contents = cast(List, contents)
        return contents
    raise Exception(
        f"Expecting a list response from {url}, instead got {json.dumps(contents)}"
    )


def get_readme_download_url(files: List[Dict[str, str]]) -> str:
    """
    Takes in a response from the github api that lists the files in a repo and
    returns the url that can be used to download the repo's README file.
    """
    for file in files:
        if file["name"].lower().startswith("readme"):
            return file["download_url"]
    return ""


def process_repo(repo: str) -> Dict[str, str]:
    """
    Takes a repo name like "gocodeup/codeup-setup-script" and returns a
    dictionary with the language of the repo and the readme contents.
    """
    contents = get_repo_contents(repo)
    readme_download_url = get_readme_download_url(contents)
    if readme_download_url == "":
        readme_contents = ""
    else:
        readme_contents = requests.get(readme_download_url).text
    return {
        "repo": repo,
        "language": get_repo_language(repo),
        "readme_contents": readme_contents,
    }


def scrape_github_data() -> List[Dict[str, str]]:
    """
    Loop through all of the repos and process them. Returns the processed data.
    """
    lst = []
    total = len(REPOS)
    for repo in REPOS:
        total -= 1
        lst.append(process_repo(repo))
        print(total)
    # return lst
    return [process_repo(repo) for repo in REPOS]


if __name__ == "__main__":
    data = scrape_github_data()
    json.dump(data, open("data2.json", "w"), indent=1)

In [33]:
scrape_github_data()

89
88
87
86
85
84
83
82
81
80
79
78
77
76
75
74
73
72
71
70
69
68
67
66
65
64
63
62
61
60
59
58
57
56
55
54
53
52
51
50
49
48
47
46
45
44
43
42
41
40
39
38
37
36
35
34
33
32
31
30
29
28
27
26
25
24
23
22
21
20
19
18
17
16
15
14
13
12
11
10
9
8
7
6
5
4
3
2
1
0


[{'repo': 'bitcoin/bitcoin',
  'language': 'C++',
  'readme_contents': "Bitcoin Core integration/staging tree\n=====================================\n\nhttps://bitcoincore.org\n\nFor an immediately usable, binary version of the Bitcoin Core software, see\nhttps://bitcoincore.org/en/download/.\n\nWhat is Bitcoin Core?\n---------------------\n\nBitcoin Core connects to the Bitcoin peer-to-peer network to download and fully\nvalidate blocks and transactions. It also includes a wallet and graphical user\ninterface, which can be optionally built.\n\nFurther information about Bitcoin Core is available in the [doc folder](/doc).\n\nLicense\n-------\n\nBitcoin Core is released under the terms of the MIT license. See [COPYING](COPYING) for more\ninformation or see https://opensource.org/licenses/MIT.\n\nDevelopment Process\n-------------------\n\nThe `master` branch is regularly built (see `doc/build-*.md` for instructions) and tested, but it is not guaranteed to be\ncompletely stable. [Tags](h